<img src="images/thro.png" align="right"> 
# A2I2 - Natural Language Processing (NLP)

## Lecture

---
## Part 2: Similarity and Similarity Search

Now, let's use our nicely cleaned Wine Review dataset to find similar wine reviews. Each wine review is a list of terms. In order to find similar wine reviews, we therefore need to define a similarity measure on lists of terms. 

## TF-IDF and Cosine Similarity

<img src="images/TF.png" align="right" width="300"> 
A common appraoch when defining a similarity measure is to map the objects into a Euclidean vector space, because there are many ways to calculate distances or similarities (e.g. the Euclidean distance).

A simple option to generate vectors from wine reviews is to arrange the terms that occur in any wine review (our vocabulary) in any order, to count the number of times each term occurs in each review, and to use this number as the value of the corresponding dimension. This is a good first step, since the count of a term (word count, or term frequency, TF) provides information on how often the term occurs in the review, i.e. presumably how important it is for the content of this review.


<img src="images/nTF.png" align="right" width="450"> 
Another aspect that we want to take into account is that wine reviews are of different lengths. The TFs (number of occurrences of a term) are therefore higher for a long review than for a short review, even if exactly the same terms occur with the same relative frequency in the two reviews. This is unattractive, but very easy to solve: we normalize the vector of each review (e.g. so that the TFs contained therein add up to 1). This results in the normalized term frequency.

We also observe that some terms appear in a large number of reviews (e.g. the term "wine" will appear in almost every review since it is not included in our stop word list), and thus provide very little help in distinguishing the reviews, while other terms appear in only a few reviews, so are probably a strong indication of the similarity of these reviews and the difference to all other reviews. We define the IDF or inverse document frequency of a term as log (N / n), where N is the number of all reviews, n the number of reviews in which the term occurs. Terms that appear in almost all reviews have an IDF of almost 0. The fewer reviews the terms is contained in, the higher its IDF value.

<img src="images/TF_IDF.png" align="right" width="310"> 
We generate the final TF-IDF vector for each review by multiplying the (normalized) TF value of the review by the IDF value of the term, i.e. weighting the term with its distinctiveness.

<img src="images/CosSim.png" align="right" width="310"> 
We could could simply use the Euclidean distance of the TF-IDF vectors as a measure of the diversity of the reviews. However, this has two drawbacks: on the one hand, it is a distance and not a similarity. On the other hand, the TF-IDF vectors are very long, i.e. the vector space is very high dimensional: there are as many dimensions as there are different terms in all reviews together. The Euclidean distance suffers from the "curse of dimensionality", i.e. the higher dimensional the vector space becomes, the less meaningful it is (the distances become more and more similar).

However, we can solve both problems elegantly by using the cosine similarity instead of the Euclidean distance. If we imagine the TF-IDF vectors as "arrows" from the origin, the direction of the arrow describes the content of the review. The angle between two such "review arrows" describes how similar the two reviews are - if the angle is 0 degrees or close to 0, if the reviews go in the same direction, if they are 90 degrees or close, the reviews are as different as possible. The angle is therefore between 0 and (absolute) 90 degrees.

If we apply the cosine function to this angle, we get a number between 1 = cos(0 degrees) for maximum similarity and 0 = cos(90 degrees) for maximum dis-similarity. This is a very nice measure of similarity, and it is also very easy to calculate: the cosine of two vectors of an Euclidean vector space is the dot product of the two divided by the product of the norms.

---
## Code

#### Setup

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import webtext
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
nltk.download('webtext')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# read the preprocessed data
with open('wines_lem.data', 'rb') as filehandle:
    wines_lem = pickle.load(filehandle)

with open('wine_lines.data', 'rb') as filehandle:
    wine_lines = pickle.load(filehandle)

# TF-IDF

In [ ]:
# compute the word counts for each document
cv=CountVectorizer(analyzer=lambda x:x)
word_count_vector=cv.fit_transform(wines_lem)
feature_names = cv.get_feature_names()

print(word_count_vector.shape)

show = 9
# get count vector for one of the documents
show_doc_vector=word_count_vector[show]

# print the count
df = pd.DataFrame(show_doc_vector.T.todense(), index=feature_names, columns=["count"])
print(wines_lem[show])
print(df.sort_values(by=["count"],ascending=False)[:10])


In [ ]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

# print the lowest and highest idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(), columns=["idf"])
print(df_idf.sort_values(by=['idf'])[:10])
print(df_idf.sort_values(by=['idf'])[-10:])

In [ ]:
# note that many of the very frequent words have low idf values, i.e. they appear in many
# reviews

In [ ]:
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(word_count_vector)

show = 0
# get tfidf vector for first document
show_doc_vector=tf_idf_vector[show]

#print the scores
df = pd.DataFrame(show_doc_vector.T.todense(), index=feature_names, columns=["tfidf"])
print(wines_lem[show])
print(df.sort_values(by=["tfidf"],ascending=False)[:20])

# Compute similar wine reviews

In [ ]:
similarities = cosine_similarity(tf_idf_vector)

In [ ]:
index = 107
df = pd.DataFrame(similarities[index], index=wine_lines, columns=["similarity"])
df['#']=np.arange(0, len(df))
df.sort_values(by=["similarity"],ascending=False)[:20]

# Word2Vec

Word2vec is a group of related models that are used to produce word embeddings. These models are shallow, two-layer neural networks that are trained to reconstruct linguistic contexts of words. Word2vec takes as its input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located close to one another in the space. [wikipedia]

In [ ]:
import gensim
import gensim.downloader as api

In [ ]:
# load a pretrained word embedding model - this one has 400.000 words with vectors of
# length 50 and has been trained on the wikipedia from 2014 plus the Gigaword 5 dataset
# see https://github.com/RaRe-Technologies/gensim-data
# and https://catalog.ldc.upenn.edu/LDC2011T07
model = api.load("glove-wiki-gigaword-50")

In [ ]:
model['wine']

In [ ]:
model.most_similar("wine")

In [ ]:
print(len(model.vocab))

In [ ]:
# remove all words not in the pre-trained vocabulary (nested list comprehension)
wines_vo = [[w for w in wine if w in model.vocab] for wine in wines_lem]

In [ ]:
# check if there are "empty" wine reviews now, i.e. reviews without any words
len([len(wine) for wine in wines_vo if len(wine)==0])

In [ ]:
# remove all these empty wine reviews (from both the word vectors and the original data)
notempty = [len(wine)>0 for wine in wines_vo]
wines_fwc = np.array(wines_vo)[notempty]
wine_lines_fwc = np.array(wine_lines)[notempty]
print(len(wines_fwc))
print(len(wine_lines_fwc))

In [ ]:
# compute the document vectors bei averaging the word vectors
rr_wv = [np.mean([model[w] for w in r if w in model.vocab], axis=0) for r in wines_fwc]

In [ ]:
# compute the cosine-similarity matrix
sim_dv = cosine_similarity(rr_wv)

In [ ]:
# find the most similar reviews for review # 100
index = 100
df = pd.DataFrame(sim_dv[index], index=wine_lines_fwc, columns=["similarity"])
df['#']=np.arange(0, len(df))
df.sort_values(by=["similarity"],ascending=False)[:20]

In [ ]:
# EOF